In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# some global variables

data_filepath = "./data/"


In [ ]:
def NONE(X):
    """Return the values - placeholder function for other operations"""
    return X

In [ ]:
# Encoding Functions

def MAP(X):
    """Map all values to integer numbers"""
    
    # WHY DOES THE MAPPING FAIL WHEN 
    # THERE IS A NAN IN THE DATA??
    # SEEMS TO HAVE WORKED BEFORE
    
    print(X)
    
    # create an encoding for categorical vars
    mapping = {label:idx for idx, label in \
               enumerate(np.unique(X))}
    
    print(mapping)
    
    return X.map(mapping).astype(int)

In [ ]:
# Scaling Functions

def NRM1(X):
    """Scale by dividing by the 1-norm"""
    norm = np.linalg.norm(X, ord=1)
    return X / norm

In [ ]:
# Imputing Functions

def UNIQ(X, value=-1):
    """Replace missing Values with unique value"""
    
    X.fillna(value=value, inplace=True)
    
    print(X)
    
    return X

In [ ]:
class Preprocessor:
    """A class to process and reformat data
    for use in learning models"""
    
    def __init__(self, train_data_file, train_label_file, 
                 test_data_file, instr_file):
        self.train_data_file = train_data_file
        self.train_label_file = train_label_file
        self.test_data_file = test_data_file
        self.instr_file = instr_file
        
    def read_data(self):
        """Reads in data from the files passed to constructor"""
        self.train_X_df= pd.read_csv(self.train_data_file)
        self.train_y_df = pd.read_csv(self.train_label_file)
        self.test_X_df = pd.read_csv(self.test_data_file)
        self.instr_df = pd.read_csv(self.instr_file)
        
    def write_data(self, out_dir='./processed_data/'):
        """Writes all of the data to output files"""
        
        # create the output directory if it does not exist
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        
        # write the data to the output directory as .csv files
        self.train_X_df.to_csv(out_dir+'train_X.csv')
        self.train_y_df.to_csv(out_dir+'train_y.csv')
        self.test_X_df.to_csv(out_dir+'test_X.csv')
        
    def process(self):
        """Performs the processing on train and test data"""
        
        for col in self.train_X_df:
            
            # determine what to perform at each of the steps
            col_instr = self.instr_df[col].values
            col_enc = col_instr[1]
            col_scl = col_instr[2]
            col_imp = col_instr[3]
            
            self.train_X_df[col] = UNIQ(self.train_X_df[col])
            
            print(self.train_X_df)

            # perform encoding of data
            if col_enc == 'MAP':
                self.train_X_df[col] = MAP(self.train_X_df[col])
            

            # perform scaling
            if col_scl == 'NRM1':
                self.train_X_df[col] = NRM1(self.train_X_df[col])


            # impute values
            if col_imp == 'UNIQ':
                self.train_X_df[col] = UNIQ(self.train_X_df[col], value=-1)

        
        self.train_X = self.train_X_df.values
        self.train_y = self.train_y_df.values
        self.test_X = self.test_X_df.values

In [ ]:
# some simple testing code and such
train_data = data_filepath+'example_data_train.csv'
train_labels = data_filepath+'example_labels_train.csv'
test_data = data_filepath+'example_data_test.csv'
description = data_filepath+'FeatureDescriptions.csv'

proc = Preprocessor(train_data_file=train_data,
                 train_label_file=train_labels,
                 test_data_file=test_data,
                 instr_file=description)

proc.read_data()

proc.process()

proc.write_data()
